Шифр Плейфера использует матрицу 5х5 (для латинского алфавита, для кириллического алфавита необходимо увеличить размер матрицы до 4х8), содержащую ключевое слово или фразу. Для создания матрицы и использования шифра достаточно запомнить ключевое слово и четыре простых правила. Чтобы составить ключевую матрицу, в первую очередь нужно заполнить пустые ячейки матрицы буквами ключевого слова (не записывая повторяющиеся символы), потом заполнить оставшиеся ячейки матрицы символами алфавита, не встречающимися в ключевом слове, по порядку (в английских текстах обычно опускается символ «Q», чтобы уменьшить алфавит, в других версиях «I» и «J» объединяются в одну ячейку). Ключевое слово может быть записано в верхней строке матрицы слева направо, либо по спирали из левого верхнего угла к центру. Ключевое слово, дополненное алфавитом, составляет матрицу 5х5 и является ключом шифра.

Для того чтобы зашифровать сообщение, необходимо разбить его на биграммы (группы из двух символов), например «Hello World» становится «HE LL OW OR LD», и отыскать эти биграммы в таблице. Два символа биграммы соответствуют углам прямоугольника в ключевой матрице. Определяем положения углов этого прямоугольника относительно друг друга. Затем, руководствуясь следующими 4 правилами, зашифровываем пары символов исходного текста:

1. Если два символа биграммы совпадают (или если остался один символ), добавляем после первого символа «Х», зашифровываем новую пару символов и продолжаем. В некоторых вариантах шифра Плейфера вместо «Х» используется «Q».
2. Если символы биграммы исходного текста встречаются в одной строке, то эти символы замещаются на символы, расположенные в ближайших столбцах справа от соответствующих символов. Если символ является последним в строке, то он заменяется на первый символ этой же строки.
3. Если символы биграммы исходного текста встречаются в одном столбце, то они преобразуются в символы того же столбца, находящиеся непосредственно под ними. Если символ является нижним в столбце, то он заменяется на первый символ этого же столбца.
4. Если символы биграммы исходного текста находятся в разных столбцах и разных строках, то они заменяются на символы, находящиеся в тех же строках, но соответствующие другим углам прямоугольника.

Для расшифровки необходимо использовать инверсию этих четырёх правил, откидывая символы «Х» (или «Q»), если они не несут смысла в исходном сообщении.

In [4]:
from math import ceil
from string import ascii_lowercase as alph
from collections import OrderedDict as OD

def reshape_list(l, m, n):
        tb = [[0 for j in range(n)] for i in range(m)]
        k = 0
        for i in range(m):
            for j in range(n):
                if (k < len(l)):
                    tb[i][j] = l[k]
                k += 1
        return tb 

def uniq_list(l):
    ul = []
    
    for elem in l:
        if elem not in ul and elem != ' ':
            ul.append(elem)
        else:
            continue
            
    return ul
    
class Playfair:
    
    def __init__(self, plaintext = "", key = ""):
        self.pt = plaintext
        self.spi = [i for i in range(len(self.pt)) if self.pt[i] == ' ']
        self.xi = [i for i in range(len(self.pt)) if self.pt[i] == 'x']
        self.k = key
        self.tb = reshape_list(uniq_list(self.k) + [c for c in alph if c not in self.k and c != 'j'], 5, 5)
        
    def info(self):
        print("Ключевая матрица: ")
        for line in self.tb:
            print(line)
        print()
            
        try:
            print("Зашифрованное сообщение: ", self.res, sep = '\n')
        except:
            print("Шифрования не произошло")
        print()
            
        try:
            print("Расшифрованное сообщение: ", self.dc, sep = '\n')
        except:
            print("Сообщение не было расшифровано")   
        print()
    
    def coord(self, x):
        for row in self.tb:
            for elem in row:
                if elem == x:
                    return [self.tb.index(row), self.tb[self.tb.index(row)].index(elem)]
        return None
    
    def cypher(self):
        tmp, i = list(self.pt.replace(' ', '')), 0
        
        while True:
            if i == len(tmp)-1:
                tmp.append('x')
            elif i > len(tmp)-1:
                break
                
            if tmp[i] == tmp[i+1]:
                tmp.insert(i + 1, 'x')
                
            c1 = self.coord(tmp[i])
            c2 = self.coord(tmp[i+1])
            if c1[0] == c2[0]:
                tmp[i] = self.tb[c1[0]][c1[1]-len(self.tb[0])+1]
                tmp[i+1] = self.tb[c1[0]][c2[1]-len(self.tb[0])+1]
            elif c1[1] == c2[1]:
                tmp[i] = self.tb[c1[0]-len(self.tb)+1][c1[1]]
                tmp[i+1] = self.tb[c2[0]-len(self.tb)+1][c1[1]]
            else:
                tmp[i] = self.tb[c1[0]][c2[1]]
                tmp[i+1] = self.tb[c2[0]][c1[1]]
            i += 2
            
        self.res = ''.join(tmp)
        
    def decypher(self):
        tmp, i = list(self.res), 0
        
        while True:
            if i >= len(tmp)-1:
                break
                
            c1 = self.coord(tmp[i])
            c2 = self.coord(tmp[i+1])
            if c1[0] == c2[0]:
                tmp[i] = self.tb[c1[0]][c1[1]-1]
                tmp[i+1] = self.tb[c1[0]][c2[1]-1]
            elif c1[1] == c2[1]:
                tmp[i] = self.tb[c1[0]-1][c1[1]]
                tmp[i+1] = self.tb[c2[0]-1][c2[1]]
            else:
                tmp[i] = self.tb[c1[0]][c2[1]]
                tmp[i+1] = self.tb[c2[0]][c1[1]]
            i += 2
        
        i = 0
        while True:
            if i > len(tmp)-1:
                break
            if tmp[i] == 'x' and i not in self.xi:
                ttmp = tmp[:i] + tmp[i+1:]
                tmp = ttmp
            i += 1
        
        for i, c in enumerate(tmp):
            if i in self.spi:
                tmp.insert(i, ' ')
        
        self.dc = ''.join(tmp)
            

#
pf = Playfair('idiocy often looks like intelligence', 'wheatson')
pf.cypher()
pf.decypher()
pf.info()
#print('kffbbzfmwaspnvcfdukdagcewpqdpnbsne')

pf = Playfair('hide the gold in the tree stump', 'playfair example')
pf.cypher()
pf.decypher()
pf.info()
#print('bmodzbxdnabekudmuixmmouvif')

pf = Playfair(input("Введите сообщение: "), input("Введите ключ: "))
pf.cypher()
pf.decypher()
pf.info()

Ключевая матрица: 
['w', 'h', 'e', 'a', 't']
['s', 'o', 'n', 'b', 'c']
['d', 'f', 'g', 'i', 'k']
['l', 'm', 'p', 'q', 'r']
['u', 'v', 'x', 'y', 'z']

Зашифрованное сообщение: 
kffbbzfmwaspnvcfdukdagcewpqdpnbsne

Расшифрованное сообщение: 
idiocy often looks like intelligence

Ключевая матрица: 
['p', 'l', 'a', 'y', 'f']
['i', 'r', 'e', 'x', 'm']
['b', 'c', 'd', 'g', 'h']
['k', 'n', 'o', 'q', 's']
['t', 'u', 'v', 'w', 'z']

Зашифрованное сообщение: 
bmodzbxdnabekudmuixmmouvif

Расшифрованное сообщение: 
hide the gold in the tree stump

Введите сообщение: extra xxx x extra
Введите ключ: asdfg
Ключевая матрица: 
['a', 's', 'd', 'f', 'g']
['b', 'c', 'e', 'h', 'i']
['k', 'l', 'm', 'n', 'o']
['p', 'q', 'r', 't', 'u']
['v', 'w', 'x', 'y', 'z']

Зашифрованное сообщение: 
mdutdvyyyydmyrpd

Расшифрованное сообщение: 
extra xxx x extra

